# Integrating Time Points and Time Intervals

In [1]:
import qualreas as qr

## References

* Allen, James F. (26 November 1983). <b>"Maintaining knowledge about temporal intervals"</b>. Communications of the ACM. 26 (11): 832–843.
    * [PDF](http://cse.unl.edu/~choueiry/Documents/Allen-CACM1983.pdf)
* Reich, Alfred J. (1994). <b>“Intervals, Points, and Branching Time.”</b> 121-133.
    * Published in TIME 1994: An International Workshop on Temporal Representation and Reasoning.
    * <b>ABSTRACT.</b> This paper extends Allen's interval algebra to include points and either left or right-branching time. The branching time algebras each contain 24 relations: Allen's original 13 relations, 5 more relations resulting from the inclusion of points, and 6 relations because of the inclusion of branching time. The paper also presents: (1) A technique for automatically deriving algebras of relations; (2) a way of representing temporal constraint networks using constraint matrices; and (3) a way of performing constraint propagation using constraint matrix multiplication.
    * <b>PDFs</b> are available at the following locations:
        * [ResearchGate](https://www.researchgate.net/publication/220810644_Intervals_Points_and_Branching_Time) (Includes a supplementary file containing algebras in structured text form.)
        * [SemanticScholar](https://www.semanticscholar.org/paper/Intervals%2C-Points%2C-and-Branching-Time-Reich/a12eca974573c812d696f3d0a58bccb171a8c39d)
        * [Time-94](http://www2.cs.uregina.ca/~temporal/time94/reich.pdf)

### An Interval & Point Algebra

Allen's original algebra of time pertains only to proper intervals.  That is, the domains and ranges of each of his 13 relations are proper time intervals.  Degenerate intervals (i.e., time points) are not permitted.

The Interval & Point algebra, described in [Reich 1994] and illustrated in this notebook, extends Allen's algebra by adding 5 new relations and modifying the domains and ranges of the relations to include, not only proper intervals, but also time points (i.e., degenerate intervals).

<p>Each of the 5 new relations have something to do with time points and so their abbreviated name starts with the letter 'P'.</p>

In [2]:
allenX = qr.Algebra("IntervalAndPointAlgebra.json")
print(sorted(allenX.relations.values(), key=lambda rel: rel.short_name))

[B, BI, D, DI, E, F, FI, M, MI, O, OI, PE, PF, PFI, PS, PSI, S, SI]


Of Allen's original 13 relations, 9 stay the same here (apply to proper intervals only), 4 have their domain, range, or both modified, and 5 are entirely new.  See the last two columns of the table in the next cell, below.

* <b>Pint</b> refers to Proper Time Intervals
* <b>Pt</b> refers to Time Points
* <b>Int</b> refers to (general) Interval, so <b>Pint</b> or <b>Pt</b>.

In [3]:
allenX.print_info()

  Algebra Name: Linear Time Interval & Point Algebra
   Description: Reich's point extension to Allen's time interval algebra (see TIME-94 paper)
 Equality Rels: [PE, E]
     Relations:
            NAME (ABBREV)         CONVERSE (ABBREV)  REFLEXIVE  SYMMETRIC TRANSITIVE  DOMAIN   RANGE
             Before (  B)               After ( BI)    False      False       True      Int     Int
              After ( BI)              Before (  B)    False      False       True      Int     Int
             During (  D)            Contains ( DI)    False      False       True      Int    PInt
           Contains ( DI)              During (  D)    False      False       True     PInt     Int
             Equals (  E)              Equals (  E)     True       True       True     PInt    PInt
           Finishes (  F)         Finished-by ( FI)    False      False       True     PInt    PInt
        Finished-by ( FI)            Finishes (  F)    False      False       True     PInt    PInt
             

Create the singleton relation sets, {Before} and {Met-By} for this algebra and compute their product

In [4]:
B = allenX.relset(["B"])
MI = allenX.relset(["MI"])
BxMI = B * MI
print(f"For the Interval & Point Algebra: {B.sorted_list()} x {MI.sorted_list()} = {BxMI.sorted_list()}")

For the Interval & Point Algebra: [B] x [MI] = []


### Multiplication is Still Not Commutative

In [5]:
MIxB = MI * B
print(MIxB.sorted_list())

BxMI = B * MI
print(BxMI.sorted_list())

print(MIxB == BxMI)

[]
[]
True


In [7]:
F = allenX.relset(["F"])

MIxF = MI * F
print(MIxF.sorted_list())

FxMI = F * MI
print(FxMI.sorted_list())

print(MIxF == FxMI)

[MI]
[BI]
False


### Equality Relations

Allen's algebra has only one identity element, E, because it deals with only one class of temporal entity, the <i>proper interval</i>.

<p>The <i>Interval and Point Algebra</i>, on the other hand, deals with two types of temporal entities, and so it has an equality relation for each one, intervals and points.  Note that intervals here can be degenerate (i.e., equivalent to a point).  This means that the relation <i>E</i> in Allen's algebra is not quite the same as the relation <i>E</i> in the Interval and Point Algebra, since the latter handles degenerate intervals, whereas the former only applies to proper intervals

<p>The equality relations of an algebra are important when propagating contraints, so there is an Algebra property that returns a list of its equality relations:

In [ ]:
print(f"The Interval & Point Algebra's equality relations: {allenX.all_equality_relations}")

## Associativity

### Example: {PS} x {B} x {D}

Associativity does not hold for this example, because <b>\(</b> {PS} \* {B} <b>\)</b> \* {D} = {D}, but PS \* <b>(</b> {B} \* {D} <b>)</b> = {B, D, PS}.

<p>The details are shown below.</p>

In [ ]:
PS = allenX.relset(["PS"])
B = allenX.relset(["B"])
D = allenX.relset(["D"])

#### First, we group {PS} & {B} together:

<p><b>(</b> {PS} x {B} <b>)</b> x {D}</p>
<p>=  {B} \* {D}</p>
<p>=  {D}</p>
<p>This calculation is illustrated below.</p>

In [ ]:
PSxB = PS * B
print "%s * %s = %s" % (PS, B, PSxB)

PSxB_x_D = PSxB * D
print "%s * %s = %s" % (PSxB, D, PSxB_x_D)

#### Then, we group {B} & {D} together:

<p>PS \* <b>(</b> {B} \* {D} <b>)</b></p>
<p>=  PS \* {B, D, M, O, PS, S}</p>
<p>=  {B, D, PS}</p>
<p>This calculation is illustrated below.</p>

In [ ]:
BxD = B * D
print BxD.sorted_list()

PS_x_BxD = PS * BxD
print PS_x_BxD.sorted_list()

#### Finally, we have...

In [ ]:
PSxB_x_D == PS_x_BxD

### So, What's Going on with Associativity?

Note on associativity: As long as all of the elements of an "algebra" have the same domain & range (and domain == range) we can talk about associativity of the algebra.  However, if the relation_lookup in the "algebra" have different domains and ranges, and those domains must match ranges in order to compose relation_lookup to find transitive relation_lookup, we're really not talking about algebras anymore.  We can't "multiply" any element in the algebra by any other element in the algebra.  For example, PS \* PS \* B does not associate.  If A{PS}B & B{PS}C ...

In [ ]:
PS = alg.relation_lookup["PS"]
print PS.domain
print PS.range

...finish this thought...

### Some Scratch Work...

Let's examine associativity for $B*D*OI$

<i>B</i> and <i>D</i> were defined earlier in this notebook:

In [ ]:
print B
print D

In [ ]:
OI = alg.relset(["OI"])
print OI

In [ ]:
BxD = B * D
DxOI = D * OI

$(B * D) * OI$ = The 13 Allen-like relation_lookup together with {PF, PS}

In [ ]:
BxD_x_OI = BxD * OI
print BxD_x_OI.sorted_list()

$B * (D * OI)$ = All 18 of the algebra's relation_lookup

In [ ]:
B_x_DxOI = B * DxOI
print B_x_DxOI.sorted_list()

So, clearly, $(B * D) * OI \ne B * (D * OI).$

Let's examine the domains and ranges involved.

In [ ]:
print alg.relation_lookup['B'].domain
print alg.relation_lookup['B'].range

In [ ]:
print alg.relation_lookup['D'].domain
print alg.relation_lookup['D'].range

In [ ]:
print alg.relation_lookup['OI'].domain
print alg.relation_lookup['OI'].range

So, chaining these together in terms of domains and ranges is:
<p>$(Int,Int) * (Int,PInt) * (PInt,PInt) = (Int, PInt)$,</p>
<p>where $Int = \{PInt, Pt\}$</p>

In [ ]:
B*D

Now, if W, X, Y, and Z are time intervals, such that W {B} X, X {D} Y, and Y {OI} Z, then...
1. In what possible ways does W relate to Y?
1. In what possible ways does X relate to Z?

To answer these questions, first look at B*D:

In [ ]:
print "%s:\n" % BxD
for rel in B*D:
    print "%s has domain %s" % (rel, list(rel.domain))

In [ ]:
print "%s:\n" % BxD
for rel in B*D:
    print "%s has range %s" % (rel, list(rel.range))

In [ ]:
W = qr.TemporalObject(['ProperInterval', 'Point'], 'W')
X = qr.TemporalObject(['ProperInterval', 'Point'], 'X')
Y = qr.TemporalObject(['ProperInterval'], 'Y')
Z = qr.TemporalObject(['ProperInterval'], 'Z')
print W
print X
print Y
print Z

In [ ]:
net_BxDxOI = qr.Network(alg, "BxDxOI")
print net_BxDxOI

Assume W, X, Y, and Z are time intervals, such that W {B} X, X {D} Y, and Y {OI} Z

In [ ]:
rel_B  = alg.relation_lookup['B']
rel_D  = alg.relation_lookup['D']
rel_OI = alg.relation_lookup['OI']

In [ ]:
net_BxDxOI.constraint(W, X, [rel_B])
net_BxDxOI.constraint(X, Y, [rel_D])
net_BxDxOI.constraint(Y, Z, [rel_OI])

In [ ]:
print net_BxDxOI

In [ ]:
net_BxDxOI.print_constraints()

In [ ]:
net_BxDxOI.propagate()

In [ ]:
net_BxDxOI.print_constraints()